In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import TensorBoard

In [2]:
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import backend as K

In [ ]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))

In [3]:
tbCallBack = TensorBoard(log_dir='./log/MNIST_CNN2', histogram_freq=0, write_graph=True, write_images=True)

In [4]:
(x_train, y_train),(x_test, y_test) = mnist.load_data()

In [5]:
K.image_data_format()

'channels_last'

In [6]:
if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, 28, 28)
    x_test = x_test.reshape(x_test.shape[0], 1, 28, 28)
    input_shape = (1, 28, 28)
else:
    x_train = x_train.reshape(x_train.shape[0], 28, 28, 1)
    x_test = x_test.reshape(x_test.shape[0], 28, 28, 1)
    input_shape = (28, 28, 1)

In [7]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train, x_test = x_train / 255.0, x_test / 255.0

In [8]:
model = Sequential([
  Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=input_shape, name="Conv1"),
  MaxPooling2D(pool_size=(2, 2), name="MaxPool1"),
  
  Conv2D(64, kernel_size=(3, 3), activation='relu', name="Conv2"),
  MaxPooling2D(pool_size=(2, 2), name="MaxPool2"),
    
  Flatten(),
  Dense(128, activation=tf.nn.relu, name="Dense1"),
  Dropout(0.2, name="Dropout1"),
  Dense(10, activation=tf.nn.softmax, name="Dense2")
])

In [9]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model = Sequential([
  Flatten(),
  Dense(512, activation=tf.nn.relu),
  Dropout(0.2),
  Dense(10, activation=tf.nn.softmax)
])

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
# Faster when batch_size=1000
#model.fit(x_train, y_train, batch_size=5000, epochs=5, validation_data=(x_test, y_test), callbacks=[tbCallBack])
model.fit(x_train, y_train, batch_size=1000, epochs=5, callbacks=[tbCallBack])

Epoch 1/5
60000/60000 [==============================] - 14s 237us/step - loss: 0.7076 - acc: 0.7888
Epoch 2/5
60000/60000 [==============================] - 12s 201us/step - loss: 0.1439 - acc: 0.9569
Epoch 3/5
60000/60000 [==============================] - 13s 215us/step - loss: 0.0892 - acc: 0.9730
Epoch 4/5
60000/60000 [==============================] - 12s 202us/step - loss: 0.0715 - acc: 0.9783
Epoch 5/5
60000/60000 [==============================] - 13s 215us/step - loss: 0.0597 - acc: 0.9818


In [11]:
model.evaluate(x_test, y_test)

10000/10000 [==============================] - 2s 209us/step


[0.04149612064701505, 0.9863]

# tensorboard --logdir log/MNIST_CNN2

In [13]:
model.save('./log/MNIST_CNN2.h5')

In [14]:
new_model = keras.models.load_model('./log/MNIST_CNN2.h5')
new_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
Conv1 (Conv2D)               (None, 26, 26, 64)        640       
_________________________________________________________________
MaxPool1 (MaxPooling2D)      (None, 13, 13, 64)        0         
_________________________________________________________________
Conv2 (Conv2D)               (None, 11, 11, 64)        36928     
_________________________________________________________________
MaxPool2 (MaxPooling2D)      (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0         
_________________________________________________________________
Dense1 (Dense)               (None, 128)               204928    
_________________________________________________________________
Dropout1 (Dropout)           (None, 128)               0         
__________

In [15]:
new_model.evaluate(x_test, y_test)

10000/10000 [==============================] - 2s 200us/step


[0.04149612064701505, 0.9863]